## Tensors
https://www.tensorflow.org/programmers_guide/variables

In [2]:
import numpy as np 
import tensorflow as tf
tf.reset_default_graph()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.01)


### Mnożenie macierzy

Uwaga: wektor to macierz $N\times1$

In [3]:
N = 81
M_local = np.random.randn(N,N).astype(np.float32)
x_local = np.random.randn(N).astype(np.float32)

In [4]:
print(M_local.nbytes/1024**3,"GB")

2.4441629648208618e-05 GB


In [5]:
with tf.device("/gpu:0"):

    M = tf.Variable(M_local, name="M")
    x = tf.Variable(x_local, name="x")

In [6]:
M = tf.Variable(tf.random_normal([N, N], stddev=0.5,dtype=tf.float64), name="M")
x = tf.Variable(tf.random_normal([N],stddev=0.1,dtype=tf.float64), name="x")

In [7]:
init_op = tf.global_variables_initializer()

In [8]:
M.get_shape(),x.get_shape(),M.dtype

(TensorShape([Dimension(81), Dimension(81)]),
 TensorShape([Dimension(81)]),
 tf.float64_ref)

In [9]:
with tf.device("/gpu:0"):
    #c = M*x
    c =  tf.matmul(M,tf.reshape(x, [ N,-1]))    
    

In [10]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True)) as sess:
    sess.run(init_op)
    M_local = sess.run(M)
    x_local = sess.run(x)
    c_local = sess.run(c)

In [11]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True)) as sess:
    sess.run(init_op)
    %time c_local = sess.run(c)
    

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2 ms


In [12]:
c_local.shape,np.dot(M_local,x_local).shape

((81, 1), (81,))

In [13]:
np.max(np.abs(np.dot(M_local,x_local) - c_local.flatten()))

1.7174966040811188

### Układ równań liniowych

In [14]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init_op)
    sol  = sess.run(tf.matrix_solve(M,tf.reshape(x, [ N,-1])))
    M_local = sess.run(M)
    x_local = sess.run(x)

In [15]:
np.ravel(sol)[:4]

array([ 0.13274714, -0.16241364,  0.14494509,  0.09861651])

In [16]:
np.linalg.solve(M_local,x_local)[:4]

array([ 0.13274714, -0.16241364,  0.14494509,  0.09861651])